# OVC SERV _ Gardening Component

### Importing the required packages

In [24]:
#Importing packages

import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from dotenv import load_dotenv

In [25]:
start_date = parse('2022-01-01')
end_date = parse('2022-03-31')

### Connecting to commcare and downloading the "comptage menage" and "All gardens" files

In [26]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('EMAIL')
password_cc = os.getenv('PASSWORD')

#Defining the driver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(1000)

#-----------------------------------------------------------------------------------------------------------------------

#Household count
def commcare_household():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/form/download/091321e0524e3ae7f5d5c1d3f43dccca/'
    )
    driver.find_element_by_xpath('//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element_by_css_selector('button[type=submit]').click()
    
commcare_household()

#Download the database "Household count"
driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-------------------------------------------------------------------------------------------------------------------------

def commcare_all_gardens():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/789629a97bddd10b4648d5138d17908e/'
    )

#All_gardens
commcare_all_gardens()

#Download the database "All gardens"
driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#--------------------------------------------------------------------------------------------------------------------------

time.sleep(5)

True



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST driver version for 99.0.4844
Driver [C:\Users\Adrien Davidson\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


### Importing all required datasets

In [4]:
#Importing the required datasets

# Parent cases for "comptage menage"
parent_cases = pd.read_excel(f"C:/Users/Adrien Davidson/Downloads/Caris Jardinage - Jaden Lakou - Comptage Menage {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Forms')

# "Comptage menage" data
household_cases = pd.read_excel(f"C:/Users/Adrien Davidson/Downloads/Caris Jardinage - Jaden Lakou - Comptage Menage {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Repeat- household_muso')

# Liste des bénéficiaires de Q1
benef_Q3 = pd.read_excel('C:/Users/Adrien Davidson/Documents/Monitoring & Evaluation/Senior M&E files/Programmes/Gardening/Gardening_ Q1FY22 Beneficiaries.xlsx', parse_dates = True)

# All gardens file with current served beneficiaries
all_gardens = pd.read_excel(f"C:/Users/Adrien Davidson/Downloads/All Gardens {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", parse_dates = True)

# Preview of the "comptage menage" data
household_cases.head(2)

,number,number__0,number__1,form.household_muso.generale_information.last_name,form.household_muso.generale_information.first_name,form.household_muso.generale_information.dob,form.household_muso.generale_information.gender,form.household_muso.generale_information.gardining_member_relationship,form.household_muso.beneficiary_less_than_18_years.hiv_test,form.household_muso.beneficiary_less_than_18_years.hiv_test_result,...,form.household_muso.beneficiary_less_than_18_years.has_been_hospitalized_during_the_past_3_months,form.household_muso.beneficiary_less_than_18_years.has_health_problem_regularly,form.household_muso.beneficiary_less_than_18_years.referred_for_a_test,form.household_muso.greather_than_or_equal_to_18.caregiver_yes_no,form.household_muso.generale_information.age_in_year,form.household_muso.generale_information.age_in_month,form.household_muso.generale_information.name,form.household_muso.beneficiary_less_than_18_years.risk_level,form.household_muso.health_id,form.household_muso.case.@case_id
0,0.0,0,0,Cherestal,Marie Fanie,1946-05-17,2,6,---,---,...,---,---,---,2,75,903,Cherestal Marie Fanie,---,NaN,bdae7bbe-5906-444a-be45-4adf3344e2ec
1,0.1,0,1,Joseph,Thomas,1938-01-28,1,6,---,---,...,---,---,---,2,83,1002,Joseph Thomas,---,NaN,ef5020cc-c5e9-4c84-b850-d62d704adfa3


In [3]:
# #Importing the required datasets

# # Parent cases for "comptage menage"
# parent_cases = pd.read_excel(f"C:/Users/Adrien Davidson/Downloads/Caris Jardinage - Jaden Lakou - Comptage Menage {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Forms')

# # "Comptage menage" data
# household_cases = pd.read_excel(f"C:/Users/Adrien Davidson/Downloads/Caris Jardinage - Jaden Lakou - Comptage Menage {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Repeat- household_muso')

# # Liste des bénéficiaires de Q3
# benef_Q3 = pd.read_excel('C:/Users/Adrien Davidson/Documents/Monitoring & Evaluation/Jardinage/Liste des bénéficiaires servis en Q3FY21/Gardening_beneficiaries_Q3FY21.xlsx')

# # All gardens file with current served beneficiaries
# all_gardens = pd.read_excel(f"C:/Users/Adrien Davidson/Downloads/All Gardens 2021-10-11.xlsx", parse_dates = True)

# # Preview of the "comptage menage" data
# household_cases.head(2)

### Data wrangling

In [5]:
#Importing dataset with hospital name, department, commune, networks name
infos = pd.read_excel('Caris Sites_modified.xlsx', usecols = ['CODE LABO', 'HOPITAL', 'RESEAU','DEPT', 'COMMUNE'],sheet_name = 'Caris Sites')
infos.rename(columns = {'CODE LABO': 'site',
                       'HOPITAL': 'hospital',
                       'DEPT': 'Department',
                       'COMMUNE': 'Commune',
                       'RESEAU': 'network'}, inplace = True)

# Cleaning
all_gardens = all_gardens[all_gardens['caris_site'] != '---']
all_gardens['caris_site'].fillna('AAAAAAAA', axis = 0, inplace = True)

#------------------------------------------------------------------------------------------------------------------------

parent_cases.rename(columns = {'number': 'number_c'}, inplace = True)
household_cases.rename(columns = {'number__0': 'number_c'}, inplace = True)

#------------------------------------------------------------------------------------------------------------------------
household_cases = household_cases.merge(parent_cases[['number_c', 'form.case.@case_id']], on = 'number_c', how = 'left')

#Creating new site and commune columns
all_gardens['site'] = all_gardens['caris_site'].apply(lambda x: x[0:8])
all_gardens.loc[(all_gardens.site == 'ZLS/STMC'), 'site'] = 'ZLS/SSPE'
all_gardens.loc[(all_gardens.site == 'GRA/CSMN'), 'site'] = 'JER/CSMN'
all_gardens.loc[(all_gardens.site == 'ART/CSAR'), 'site'] = 'GON/CSAR'
all_gardens.loc[(all_gardens.site == 'GRA/HSPC'), 'site'] = 'JER/HSPC'
all_gardens.loc[(all_gardens.site == 'ZLS/POZS'), 'site'] = 'MON/POZS'

all_gardens = all_gardens.merge(infos[['site', 'Commune']], on = 'site', how = 'left')

#Function to get the age range
def age_range(age):
    if age < 1:
        return '< 1'
    elif age in range(1, 5):
        return '1 - 4'
    elif age in range(5, 10):
        return '5 - 9'
    elif age in range(10, 15):
        return '10 - 14'
    elif age in range(15, 18):
        return '15 - 17'
    else:
        return '18+'


#Creating age range in all gardens
all_gardens['age'] = pd.to_numeric(all_gardens['age'])
all_gardens['age_range'] = all_gardens['age'].map(age_range)

### Cleaning the Q3 beneficiaries dataset

In [6]:
# Removing the Caris, nutrition, dreams and muso beneficiaries
benef_dedup_Q3 = benef_Q3[~(benef_Q3['beneficiary_type'].isin(['Caris', 'malnutrition_program', 'muso', 'dreams'])) & (benef_Q3['last_name'] != '---')][['info.case_id']]
print(f"Number of beneficiaries for Q3: {benef_dedup_Q3.shape[0]}.")

Number of beneficiaries for Q3: 2472.


### Number of beneficiaries for Q3 and Q4

In [7]:
#Beneficiaries served in both quarters

all_gardens['info.last_modified_date'] = pd.to_datetime(all_gardens['info.last_modified_date'])
benef_Q3_Q4 = all_gardens[(all_gardens['info.case_id'].isin(benef_dedup_Q3['info.case_id'])) 
                          & ((all_gardens['info.last_modified_date'] >= start_date) 
                             & (all_gardens['info.last_modified_date'] <= end_date))
                         & (all_gardens['closed'] == False)]
print(f"Number of beneficiaries served in Q3 AND Q4: {benef_Q3_Q4.shape[0]}.")

Number of beneficiaries served in Q3 AND Q4: 2359.


In [21]:
serv1 = benef_dedup_Q3[['info.case_id']]
serv2 = benef_Q3_Q4[['info.case_id', 'beneficiary_type']]

serv1.shape[0]
serv2.shape[0]

dt = serv1.merge(serv2, on = 'info.case_id', how = 'left').fillna('---')
dt

2472

2359

,info.case_id,beneficiary_type
0,5b8dc5a8-fe8b-4405-92d5-ca99e1e8c57b,neighbor
1,a6e08ae4-3ebb-42fa-94c3-edae1782d091,neighbor
2,204a646d-1b93-46a4-9832-03cb9ddf1be7,neighbor
3,059ea1a5-b72e-41b7-b0e0-b3f8ed1e7d74,neighbor
4,24d69fdf-30aa-42c3-a6f7-ec71b026cbca,neighbor
...,...,...
2467,265f0814-3e8b-497e-8f13-ec08033d3f09,---
2468,2e4c39ca-488c-4a08-8b22-2927efc33511,---
2469,b8756337-8ba6-494c-95d2-4a4dde5855e4,---
2470,587bd9cc-fcf0-410a-9a7a-ef0148464f3c,neighbor


In [23]:
dt.to_excel('113.xlsx', index = False)

In [12]:
#New beneficiaries
all_gardens['start_date'] = pd.to_datetime(all_gardens['start_date'], errors = 'coerce')

new_benef = all_gardens[~(all_gardens['beneficiary_type'].isin(['Caris', 'malnutrition_program', 'muso', 'dreams'])) & (all_gardens['start_date'] >= start_date) & (all_gardens['start_date'] <= end_date)]
print(f"Number of new beneficiaries: {new_benef.shape[0]}")

Number of new beneficiaries: 752


In [14]:
# new_benef.to_excel('Data_checking_new.xlsx', index = False)

In [13]:
OVC = pd.concat([benef_Q3_Q4, new_benef]).drop_duplicates('info.case_id').reset_index(drop = True)
OVC = OVC[OVC.age_range == '18+']
print(f"Direct OVC Gardening: {OVC.shape[0]}")

Direct OVC Gardening: 3099


In [12]:
#Export the reportable beneficiaries
OVC.to_excel("Gard_OVC_Q2FY22_n.xlsx", index = False)

# Comptage

In [16]:
#Comptage pour les benefs de Q3_Q4
parent_cases['form.household_number'] = pd.to_numeric(parent_cases['form.household_number'])
hhg_Q3_Q4 = parent_cases[parent_cases['form.case.@case_id'].isin(OVC['info.case_id'])][['form.case.@case_id', 'form.household_number']]
print(f"Number of beneficiaries counted: {len(hhg_Q3_Q4['form.case.@case_id'].unique())}.")

Number of beneficiaries counted: 2923.


In [14]:
hhg_Q3_Q4.to_excel("Gard_count_Q2FY22_n.xlsx", index = False)

# DATIM

In [17]:
household_cases.rename(columns = {'form.case.@case_id': 'info.case_id'}, inplace = True)
household_cases = household_cases.merge(all_gardens[['info.case_id', 'Commune']], on = 'info.case_id', how = 'left')
household_cases.head(2)

,number,number_c,number__1,form.household_muso.generale_information.last_name,form.household_muso.generale_information.first_name,form.household_muso.generale_information.dob,form.household_muso.generale_information.gender,form.household_muso.generale_information.gardining_member_relationship,form.household_muso.beneficiary_less_than_18_years.hiv_test,form.household_muso.beneficiary_less_than_18_years.hiv_test_result,...,form.household_muso.beneficiary_less_than_18_years.referred_for_a_test,form.household_muso.greather_than_or_equal_to_18.caregiver_yes_no,form.household_muso.generale_information.age_in_year,form.household_muso.generale_information.age_in_month,form.household_muso.generale_information.name,form.household_muso.beneficiary_less_than_18_years.risk_level,form.household_muso.health_id,form.household_muso.case.@case_id,info.case_id,Commune
0,0.0,0,0,Cherestal,Marie Fanie,1946-05-17,2,6,---,---,...,---,2,75,903,Cherestal Marie Fanie,---,NaN,bdae7bbe-5906-444a-be45-4adf3344e2ec,967a0f80-aad5-450a-8e7f-f21425035682,Cayes
1,0.1,0,1,Joseph,Thomas,1938-01-28,1,6,---,---,...,---,2,83,1002,Joseph Thomas,---,NaN,ef5020cc-c5e9-4c84-b850-d62d704adfa3,967a0f80-aad5-450a-8e7f-f21425035682,Cayes


In [18]:
#preparing age columns
household_cases.loc[(household_cases['form.household_muso.generale_information.age_in_year'] == '---'), 'form.household_muso.generale_information.age_in_year'] = ''
household_cases['form.household_muso.generale_information.age_in_year'] = pd.to_numeric(household_cases['form.household_muso.generale_information.age_in_year'])
household_cases['age_range'] = household_cases['form.household_muso.generale_information.age_in_year'].map(age_range)

#Replacing code by true gender values
household_cases.loc[(household_cases['form.household_muso.generale_information.gender'] == str(1)), 'form.household_muso.generale_information.gender'] = 'Male'
household_cases.loc[(household_cases['form.household_muso.generale_information.gender'] == str(2)), 'form.household_muso.generale_information.gender'] = 'Female'

household_cases.head(2)

,number,number_c,number__1,form.household_muso.generale_information.last_name,form.household_muso.generale_information.first_name,form.household_muso.generale_information.dob,form.household_muso.generale_information.gender,form.household_muso.generale_information.gardining_member_relationship,form.household_muso.beneficiary_less_than_18_years.hiv_test,form.household_muso.beneficiary_less_than_18_years.hiv_test_result,...,form.household_muso.greather_than_or_equal_to_18.caregiver_yes_no,form.household_muso.generale_information.age_in_year,form.household_muso.generale_information.age_in_month,form.household_muso.generale_information.name,form.household_muso.beneficiary_less_than_18_years.risk_level,form.household_muso.health_id,form.household_muso.case.@case_id,info.case_id,Commune,age_range
0,0.0,0,0,Cherestal,Marie Fanie,1946-05-17,Female,6,---,---,...,2,75.0,903,Cherestal Marie Fanie,---,NaN,bdae7bbe-5906-444a-be45-4adf3344e2ec,967a0f80-aad5-450a-8e7f-f21425035682,Cayes,18+
1,0.1,0,1,Joseph,Thomas,1938-01-28,Male,6,---,---,...,2,83.0,1002,Joseph Thomas,---,NaN,ef5020cc-c5e9-4c84-b850-d62d704adfa3,967a0f80-aad5-450a-8e7f-f21425035682,Cayes,18+


In [19]:
hh_gard = household_cases[household_cases['info.case_id'].isin(hhg_Q3_Q4['form.case.@case_id'])]
print(f"Household members: {hh_gard.shape[0]}.")

Household members: 12014.


In [48]:
hh_gard.to_excel('Comptage_Gardening_FY22Q2.xlsx', index = False)

In [20]:
#---------------------------------------------------------------------------------------------------------------------
hh_gard.rename(columns = {'form.household_muso.generale_information.gender': 'gender'}, inplace = True)
#---------------------------------------------------------------------------------------------------------------------


#Creating Datim Table for mothers' kids
datim_gard = hh_gard.pivot_table(index = 'Commune', columns = ['gender', 'age_range'], values = 'info.case_id', 
                              aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18+'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18+')]
datim_gard = datim_gard.reindex(columns = columns)
datim_gard[('Female', '18+')] = 0
datim_gard[('Male', '18+')] = 0
datim_gard.reset_index(inplace = True)
datim_gard

gender                Commune Female                                 Male  \
age_range                        < 1 1 - 4 5 - 9 10 - 14 15 - 17 18+  < 1   
0             Anse D'Hainault      3    30    19      25      16   0    5   
1                  Anse-Rouge      0     0     0       1       0   0    0   
2               Anse-a-Galets      0     1     0       0       0   0    0   
3                    Arcahaie      0     0     0       1       0   0    0   
4               Baie-de-Henne      4    11     8       9       9   0    0   
5                 Bassin Bleu      0     0     2       4       0   0    0   
6               Bombardopolis      0     2     2       5       1   0    1   
7                 Camp-Perrin      0     2     4       6       3   0    0   
8                 Cap-Haitien      0     0     2       0       0   0    0   
9                   Capotille      6    16    27      30      18   0    7   
10                  Cavaillon      2     6     0       6       5   0    0   
11                      Cayes      1     4    10      13       5   0    1   
12                     Corail      5     4     5      13       7   0    1   
13                 Dessalines      2     6    10       8       9   0    1   
14                     Ennery      2     2     1       3       2   0    1   
15                   Gonaives      2     6     8       5       1   0    3   
16                    Jeremie      3    45    49      64      49   0    3   
17                   Kenscoff      0     4     3       0       1   0    0   
18                   L'Estere      2     9     9       5       5   0    1   
19                Les Anglais      1    11    17      18      10   0    0   
20                      Limbe      4    16    23      23      19   0    0   
21                   Limonade      2     1     2       4       2   0    0   
22                    Marigot      0     0     0       2       1   0    0   
23                  Marmelade      0     1     2       8      12   0    1   
24                      Milot     11    95   116     103      60   0   27   
25              Mombin-Crochu     23    47    75      73      45   0   13   
26              Mont-Organise      8    38    51      41      15   0    8   
27                      Moron      3    12    27      26      14   0    1   
28                Ouanaminthe      3    19    30      31      14   0    3   
29                     Pignon      4    27    37      30      27   0    3   
30                     Pilate     11    47    66      95      48   0   19   
31             Plaine-du-Nord      0    11    12      11       1   0    0   
32                Port-Margot      4    53   105     106      45   0    2   
33                 Port-Salut      0    26    24      29      18   0    6   
34               Port-de-Paix      1     4     9      14       2   0    1   
35         Saint-Louis du Sud      0     0     0       0       0   0    0   
36              Terrier-Rouge      2     6     5      16       7   0    1   
37                    Thiotte      0     0     0       0       0   0    0   
38               Trou-du-Nord      6    22    13      32      15   0    1   
39                      Total    115   584   773     860     486   0  110   

gender                                     
age_range 1 - 4 5 - 9 10 - 14 15 - 17 18+  
0            22    37      39      21   0  
1             0     0       0       0   0  
2             0     0       0       0   0  
3             0     0       1       0   0  
4            13     6      10       7   0  
5             0     0       1       0   0  
6             6     4       0       2   0  
7             2     3       3       2   0  
8             1     0       0       0   0  
9            16    14      34      15   0  
10            7     6       1       0   0  
11            8     8       5       7   0  
12            9    11      13       2   0  
13            4     8      11      12   0  
14            0     2       2       2   0  
15            6  

# Caregivers

In [21]:
hh_gard.rename(columns = {'form.household_muso.greather_than_or_equal_to_18.caregiver_yes_no': 'caregivers'}, inplace = True)

hhg_caregivers = hh_gard[(hh_gard['age_range'] == '18+') & (hh_gard['caregivers'] == str(1))]
print(f"Number of caregivers: {hhg_caregivers.shape[0]}.")

Number of caregivers: 3319.


In [22]:
#Clustering by commune
caregiver = hhg_caregivers.pivot_table(index = 'Commune', values = 'info.case_id', columns = ['gender', 'caregivers'],
                                      aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
caregiver.drop('Total', axis = 1, inplace = True)
caregiver.rename(columns = {'1': 'Caregiver 18+'}, inplace = True)
caregiver.reset_index(inplace = True)
caregiver

gender,Commune,Female,Male
caregivers,,Caregiver 18+,Caregiver 18+
0,Anse D'Hainault,22,75
1,Anse-a-Galets,0,1
2,Arcahaie,0,1
3,Baie-de-Henne,47,21
4,Bassin Bleu,13,14
5,Bombardopolis,10,8
6,Cap-Haitien,0,1
7,Capotille,7,55
8,Cavaillon,3,11


# Total indirect members

In [23]:
print(f"""Nombre total de bénéficiaires indirects à rapporter:
      {hh_gard[hh_gard['age_range'] != '18+'].shape[0] + hhg_caregivers.shape[0]}""")

Nombre total de bénéficiaires indirects à rapporter:
      8794


# OVC_HIVSTAT

In [18]:
#Replacing
hh_gard.rename(columns = {'form.household_muso.generale_information.age_in_year': 'age_in_year'}, inplace = True)
hh_gard.rename(columns = {'form.household_muso.beneficiary_less_than_18_years.risk_level': 'risk_level'}, inplace = True)
hh_gard.rename(columns = {'form.household_muso.beneficiary_less_than_18_years.hiv_test_result': 'Test_result'}, inplace = True)

hh_gard.loc[(hh_gard['form.household_muso.beneficiary_less_than_18_years.hiv_test'] == str(1)), 'form.household_muso.beneficiary_less_than_18_years.hiv_test'] = 'oui'
hh_gard.loc[(hh_gard['form.household_muso.beneficiary_less_than_18_years.hiv_test'] == str(2)), 'form.household_muso.beneficiary_less_than_18_years.hiv_test'] = 'non'
hh_gard.loc[(hh_gard['form.household_muso.beneficiary_less_than_18_years.hiv_test'] == str(3)), 'form.household_muso.beneficiary_less_than_18_years.hiv_test'] = 'Ne sait pas'

hh_gard.rename(columns = {'form.household_muso.beneficiary_less_than_18_years.is_on_arv': 'on_arv'}, inplace = True)
hh_gard.loc[(hh_gard['on_arv'] == str(1)), 'on_arv'] = 'oui'
hh_gard.loc[(hh_gard['on_arv'] == str(2)), 'on_arv'] = 'non'
hh_gard.loc[(hh_gard['on_arv'] == str(3)), 'on_arv'] = 'ne sait pas'
#
positive_children = hh_gard[(hh_gard['Test_result'] == 'positif') & (hh_gard['age_range'] != '18+')]
negative_children = hh_gard[(hh_gard['form.household_muso.beneficiary_less_than_18_years.hiv_test'] == 'oui') & (hh_gard['age_in_year'] < 18) & (hh_gard['Test_result'] == 'negatif')]

positive_children.shape[0]
negative_children.shape[0]

1

14

In [19]:
#Positive children
positive_children.rename(columns = {'form.household_muso.beneficiary_less_than_18_years.hiv_test_result': 'Test_result'}, inplace = True)
hhg_positive_children = positive_children.pivot_table(index = 'Commune', values = 'info.case_id', columns = 'Test_result',
                                                     aggfunc = len, fill_value = 0)
hhg_positive_children.reset_index(inplace = True)
hhg_positive_children.rename(columns = {'positif': 'positive'}, inplace = True)
hhg_positive_children

Test_result,Commune,positive
0,Gonaives,1


In [20]:
#Negative children
negative_children.rename(columns = {'form.household_muso.beneficiary_less_than_18_years.hiv_test_result': 'Test_result'}, inplace = True)
hhg_negative_children = negative_children.pivot_table(index = 'Commune', values = 'info.case_id', columns = 'Test_result',
                                                     aggfunc = len, fill_value = 0)
hhg_negative_children.reset_index(inplace = True)
hhg_negative_children.rename(columns = {'negatif': 'negative'}, inplace = True)
hhg_negative_children.head()

Test_result,Commune,negative
0,Anse D'Hainault,2
1,Kenscoff,2
2,Milot,5
3,Port-Margot,2
4,Port-Salut,3


In [21]:
np.sum(hhg_negative_children.negative)

14

In [22]:
#Children on arv

children_on_arv = positive_children[positive_children['on_arv'] == 'oui']
children_on_arv = children_on_arv.pivot_table(index = 'Commune', columns = 'on_arv', values = 'info.case_id',
                                                 aggfunc = len, fill_value = 0)
children_on_arv.rename(columns = {'oui': 'on_arv'}, inplace = True)
children_on_arv.reset_index(inplace = True)
children_on_arv

,index


In [23]:
#Children not on arv

children_not_on_arv = positive_children[(pd.isnull(positive_children.on_arv)) | (positive_children['on_arv'] != 'oui')]
hhg_children_not_on_arv = children_not_on_arv.groupby('Commune').count()[['info.case_id']]
hhg_children_not_on_arv.reset_index(inplace = True)
hhg_children_not_on_arv.rename(columns = {'info.case_id': 'not_on_arv'}, inplace = True)
hhg_children_not_on_arv

,Commune,not_on_arv
0,Gonaives,1


In [24]:
#No HIV reported (Status Unknown)
no_status = hh_gard[(hh_gard['age_in_year'] < 18) & (hh_gard['risk_level'] == str(1)) & (~hh_gard['Test_result'].isin(['positif', 'negatif']))]
children_unknown_status = no_status.pivot_table(index = 'Commune', columns = 'risk_level', values = 'info.case_id',
                                     aggfunc = len, fill_value = 0)
children_unknown_status.rename(columns = {'1':'No_HIV_reported_Status_Unknown'}, inplace = True)
children_unknown_status.reset_index(inplace = True)
children_unknown_status.head()

risk_level,Commune,No_HIV_reported_Status_Unknown
0,Anse D'Hainault,1
1,Baie-de-Henne,6
2,Bombardopolis,3
3,Capotille,1
4,Cavaillon,1


In [25]:
np.sum(children_unknown_status.No_HIV_reported_Status_Unknown)

69

In [26]:
#Test not indicated
not_indicated = hh_gard[(hh_gard['age_in_year'] < 18) & (hh_gard['risk_level'] == str(0)) & (~hh_gard['Test_result'].isin(['positif', 'negatif']))]
hhg_not_indicated = not_indicated.pivot_table(index = 'Commune', columns = 'risk_level', values = 'info.case_id',
                                     aggfunc = len, fill_value = 0)
hhg_not_indicated.rename(columns = {'0':'test not indicated'}, inplace = True)
hhg_not_indicated.reset_index(inplace = True)
hhg_not_indicated

risk_level,Commune,test not indicated
0,Anse D'Hainault,38
1,Anse-a-Galets,1
2,Arcahaie,2
3,Baie-de-Henne,51
4,Bassin Bleu,8
5,Bombardopolis,18
6,Cap-Haitien,3
7,Capotille,86
8,Cavaillon,32
9,Cayes,59


In [27]:
np.sum(hhg_not_indicated['test not indicated'])

4136

In [28]:
index = ['Commune','positive', 'on_arv', 'not_on_arv', 'negative', 'test not indicated', 'No_HIV_reported_Status_Unknown']
#------------------------------------------------------------------------------------------------------------------------
def ovc_hivstat_children_jardinage():
    if len(hhg_positive_children) != 0 and len(children_on_arv) != 0 and len(hhg_children_not_on_arv) != 0 and len(hhg_negative_children) != 0 and len(children_unknown_status) != 0 and len(hhg_not_indicated) != 0:
        ovc_hivstat_children = hhg_positive_children.merge(children_on_arv, on = 'Commune', how = 'outer').merge(hhg_children_not_on_arv, on = 'Commune', how = 'outer').merge(hhg_negative_children, on = 'Commune', how = 'outer').merge(hhg_not_indicated, on = 'Commune', how = 'outer').merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(hhg_positive_children) != 0 and len(children_on_arv) == 0 and len(hhg_children_not_on_arv) != 0 and len(hhg_negative_children) != 0 and len(children_unknown_status) != 0 and len(hhg_not_indicated) != 0:
        ovc_hivstat_children = hhg_positive_children.merge(hhg_children_not_on_arv, on = 'Commune', how = 'outer').merge(hhg_negative_children, on = 'Commune', how = 'outer').merge(hhg_not_indicated, on = 'Commune', how = 'outer').merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['children_on_arv'] = 0
        ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(hhg_positive_children) != 0 and len(children_on_arv) != 0 and len(hhg_children_not_on_arv) == 0 and len(hhg_negative_children) != 0 and len(children_unknown_status) != 0 and len(hhg_not_indicated) != 0:
        ovc_hivstat_children = hhg_positive_children.merge(children_on_arv, on = 'Commune', how = 'outer').merge(hhg_negative_children, on = 'Commune', how = 'outer').merge(hhg_not_indicated, on = 'Commune', how = 'outer').merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['hhg_children_not_on_arv'] = 0
        ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(hhg_positive_children) == 0 and len(hhg_negative_children) != 0 and len(children_unknown_status) != 0 and len(hhg_not_indicated) != 0:
        ovc_hivstat_children = hhg_negative_children.merge(hhg_not_indicated, on = 'Commune', how = 'outer').merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children[['positive','on arv', 'not on arv']] = 0
        ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)
        return ovc_hivstat_children
    elif len(hhg_positive_children) == 0 and len(hhg_negative_children) == 0 and len(children_unknown_status) != 0 and len(hhg_not_indicated) != 0:
        ovc_hivstat_children = hhg_not_indicated.merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children[['positive','on arv', 'not on arv', 'negative']] = 0
        ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)
        return ovc_hivstat_children
    else:
        return 'HIV test data are not available right now!!'
    
    
#Calling the function
ovc_hivstat_children_jardinage()

,Commune,positive,on_arv,not_on_arv,negative,test not indicated,No_HIV_reported_Status_Unknown
0,Anse D'Hainault,NaN,NaN,NaN,2.0,38,1.0
1,Anse-a-Galets,NaN,NaN,NaN,NaN,1,NaN
2,Arcahaie,NaN,NaN,NaN,NaN,2,NaN
3,Baie-de-Henne,NaN,NaN,NaN,NaN,51,6.0
4,Bassin Bleu,NaN,NaN,NaN,NaN,8,NaN
5,Bombardopolis,NaN,NaN,NaN,NaN,18,3.0
6,Cap-Haitien,NaN,NaN,NaN,NaN,3,NaN
7,Capotille,NaN,NaN,NaN,NaN,86,1.0
8,Cavaillon,NaN,NaN,NaN,NaN,32,1.0
9,Cayes,NaN,NaN,NaN,NaN,59,3.0


# Direct beneficiaries

In [46]:
# Number of direct beneficiaries older than 17
direct_benef = OVC[['info.case_id', 'Commune', 'age_range']]
direct_benef['gender'] = 'Female'
direct_benef = direct_benef[direct_benef['age_range'] == '18+']
direct_benef.head()

,info.case_id,Commune,age_range,gender
0,5b8dc5a8-fe8b-4405-92d5-ca99e1e8c57b,Ouanaminthe,18+,Female
1,a6e08ae4-3ebb-42fa-94c3-edae1782d091,Ouanaminthe,18+,Female
2,204a646d-1b93-46a4-9832-03cb9ddf1be7,Ouanaminthe,18+,Female
3,059ea1a5-b72e-41b7-b0e0-b3f8ed1e7d74,Ouanaminthe,18+,Female
4,24d69fdf-30aa-42c3-a6f7-ec71b026cbca,Ouanaminthe,18+,Female


In [47]:
dbenef = direct_benef.pivot_table(index = 'Commune', values = 'info.case_id', columns = ['gender', 'age_range'],
                                       aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
dbenef.drop('Total', axis = 1, inplace = True)
dbenef.reset_index(inplace = True)
dbenef

gender,Commune,Female
age_range,,18+
0,Anse D'Hainault,122
1,Anse-Rouge,1
2,Anse-a-Galets,1
3,Arcahaie,1
4,Baie-de-Henne,61
5,Bassin Bleu,23
6,Bombardopolis,14
7,Camp-Perrin,15
8,Cap-Haitien,1


# Export

In [31]:
import openpyxl

with pd.ExcelWriter('Services_Jardinage.xlsx', engine = 'openpyxl') as writer:
    benef_Q3_Q4.to_excel(writer, sheet_name = 'Bénéficiaires_Q3_Q4', index = False)
    new_benef.to_excel(writer, sheet_name = 'Bénéficiaires_Q4', index = False)


with pd.ExcelWriter("DATIM_Jardinage.xlsx", engine = 'openpyxl') as writer:
    dbenef.to_excel(writer, sheet_name = 'Bénéficiaires directs', na_rep = 0)
    datim_gard.to_excel(writer, sheet_name = 'Datim')
    ovc_hivstat_children_jardinage().to_excel(writer, sheet_name = 'HIVSTAT', index = False, na_rep = 0)
    caregiver.to_excel(writer, sheet_name = 'Caregivers')

# Closing the connection ...

In [10]:
driver.quit()

---